<a href="https://colab.research.google.com/github/ekdnam/GSoC_HSF_ATLAS/blob/main/EDA_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!rm -rf GSoC_HSF_ATLAS
!git clone https://github.com/ekdnam/GSoC_HSF_ATLAS
!cd GSoC_HSF_ATLAS

Cloning into 'GSoC_HSF_ATLAS'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 2), reused 18 (delta 2), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [12]:
!ls

GSoC_HSF_ATLAS	sample_data


In [3]:
import sys
print(sys.version)

3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from sklearn.preprocessing import StandardScaler

In [9]:
class DatasetReader:
    def __init__(self, filename: str = "dataset/dataset.csv", is_colab: bool = True):
        self.filename = filename
        self.repo_name = 'GSoC_HSF_ATLAS/'
        self.filepath = self.repo_name + filename if is_colab else filename
        self.cols = ["event_ID", "process_ID", "event_weight", "MET", "MET_Phi"]
        self.ignore_particles = ["e-", "e+", "m-", "m+", "g", "b"]
        self.data_list = []

    def readFile(self) -> List[List[str]]:
        print("Opening file: " + self.filepath)
        file = open(self.filepath, "r")

        print("Reading file line by line...")
        for line in file.readlines():
            cleaned_line = line.replace(";", ",")
            cleaned_line = cleaned_line.rstrip(",\n")
            cleaned_line = cleaned_line.split(",")
            self.data_list.append(cleaned_line)
        return self.data_list

    def createDataFrame(self):
        longest_line = max(self.data_list, key=len)

        n_max_cols = len(longest_line)
        print("Number of maximum possible columns: " + str(n_max_cols))

        print("Our cols are: " + str(self.cols))
        print("Creating deep copy of cols")
        copy_cols = self.cols.copy()

        for i in range(1, (int((n_max_cols - 5) / 5)) + 1):
            self.cols.append("obj_" + str(i))
            self.cols.append("E_" + str(i))
            self.cols.append("pt_" + str(i))
            self.cols.append("eta_" + str(i))
            self.cols.append("phi_" + str(i))

        print("Number of cols: " + str(len(self.cols)))
        print("\nSlicing list of cols: " + str(self.cols[50:60]))

        df = pd.DataFrame(self.data_list, columns=self.cols)
        df.fillna(value=np.nan, inplace=True)

#         return df
        df_data = pd.DataFrame(df.values, columns=self.cols)
        df_data.fillna(value=0, inplace=True)
        df_data.drop(columns=copy_cols, inplace=True)

        ignore_list = []
        for i in range(len(df_data)):
            for j in df_data.loc[i].keys():
                if "obj" in j:
                    if df_data.loc[i][j] in self.ignore_particles:
                        ignore_list.append(i)
                        break

        df_data.drop(ignore_list, inplace=True)

        x = df_data.values.reshape([df_data.shape[0] * df_data.shape[1] // 5, 5])

        temp_list = []
        for i in range(x.shape[0]):
            if (x[i] == 0).all():
                temp_list.append(i)        
        x1 = np.delete(x, temp_list, 0)
        del x

        temp_list = []
        for i in range(x1.shape[0]):   
            if  (x1[i][0] == 'j'):
                continue
            else:
                temp_list.append(i)
                print(i, x1[i][0])
        
        data = np.delete(x1, temp_list, 0)

        col_names = ['obj', 'E', 'pt', 'eta', 'phi']
        data_df = pd.DataFrame(data, columns=col_names)
        # Drop the 'obj' column as it's unnecessary
        data_df.drop(columns='obj', inplace=True)
        data_df = data_df.astype('float32')

        return data_df

In [10]:
dr = DatasetReader(is_colab = True)
data = dr.readFile()
df = dr.createDataFrame()

Opening file: GSoC_HSF_ATLAS/dataset/dataset.csv


FileNotFoundError: ignored